In [4]:
"""
This script applies the linear local recession curve watertable fluctuation method (e.g., Wilopo and Putra 2021) to estimate recharge from synthetic hydrographs. The method is implemented on a series of generated 
recharge events, and recharge estimates are compared to known values. The output is the relative error (%) of the 
recharge estimate, calculated as described in Becke et al. (2025).
"""
# Import packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
from pathlib import Path

# --- Configuration ---
FOLDER_NAME = "Linear_LRC"
BASE_DIR = Path.home() / "workspace" / "WTF" / "OutputFiles" / 
OUTPUT_DIR = BASE_DIR / FOLDER_NAME
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

INPUT_DIR = BASE_DIR / "data" 

# Time setup
time = np.linspace(0, 400., 401)
event_windows = {
    1: {"fit": (30, 90), "extend": (30, 120), "check": 120},
    2: {"fit": (120, 210), "extend": (120, 240), "check": 240},
    3: {"fit": (240, 360), "extend": (240, 390), "check": 390},
}

def objective_linear(x, C1, C2):
    return C1 * x + C2

def process_linear_errors(itr, event_id, time, fit_range, extend_range, check_index):
    x_fit = time[fit_range[0]:fit_range[1] + 1]
    x_extend = time[extend_range[0]:extend_range[1] + 1]

    errors = []

    for j in range(1, itr + 1):
        try:
            wl_file = INPUT_DIR / f"hydro_{j:04d}.csv"
            var_file = INPUT_DIR / f"var_{j:04d}.csv"

            wl = pd.read_csv(wl_file, header=None).to_numpy().flatten()
            var = pd.read_csv(var_file, header=None).to_numpy()

            y_fit = wl[fit_range[0]:fit_range[1] + 1]

            # Fit linear model
            params, _ = curve_fit(objective_linear, x_fit, y_fit, absolute_sigma=True, maxfev=50000, method='trf')
            C1, C2 = params

            y_projected = objective_linear(x_extend, C1, C2)

            delta_h_emp = wl[check_index] - y_projected[-1]
            delta_t = x_extend[-1] - x_fit[-1]
            Sy = var[3][0]
            true_rech = var[0][0]

            rech_est = (delta_h_emp / delta_t) * Sy
            rech_error = ((rech_est - true_rech) / true_rech) * 100

            errors.append(rech_error)

            #print(f"[RE{event_id}] File {j:04d} | y = {C1:.5f} * x + {C2:.5f} | Recharge error = {rech_error:.2f}%")

        except Exception as e:
            print(f"[Warning] Skipping hydrograph {j:04d}: {e}")
            continue

    return errors

# --- Run for all events ---
for event_id, params in event_windows.items():
    errors = process_linear_errors(
        itr=1000,
        event_id=event_id,
        time=time,
        fit_range=params["fit"],
        extend_range=params["extend"],
        check_index=params["check"]
    )

    output_file = OUTPUT_DIR / f"RE{event_id}_Linear_Rerror.csv"
    np.savetxt(output_file, errors, delimiter=',', header=f'Linear_Rerror_Event{event_id}', comments='')
    print(f"Saved results to {output_file}")


[RE1] File 0001 | y = -0.00071 * x + 0.12275 | Recharge error = 2.55%
[RE1] File 0002 | y = -0.00179 * x + 1.68544 | Recharge error = -3.89%
[RE1] File 0003 | y = -0.00541 * x + 0.71714 | Recharge error = 7.99%
[RE1] File 0004 | y = -0.00719 * x + 0.92330 | Recharge error = -57.41%
[RE1] File 0005 | y = -0.00698 * x + 1.03526 | Recharge error = -8.06%
[RE1] File 0006 | y = -0.08148 * x + 14.20064 | Recharge error = 2.49%
[RE1] File 0007 | y = -0.00020 * x + 2.66262 | Recharge error = -1.25%
[RE1] File 0008 | y = -0.00260 * x + 0.25522 | Recharge error = 0.73%
[RE1] File 0009 | y = -0.00320 * x + 2.08190 | Recharge error = -5.39%
[RE1] File 0010 | y = -0.00252 * x + 1.42049 | Recharge error = -5.75%
Saved results to C:\Users\beck0213\workspace\WTF\OutputFiles\GITHUB_CHECK\Linear_LRC\RE1_Linear_Rerror.csv
[RE2] File 0001 | y = -0.00069 * x + 0.23359 | Recharge error = 4.32%
[RE2] File 0002 | y = -0.00581 * x + 3.74957 | Recharge error = -2.45%
[RE2] File 0003 | y = -0.00584 * x + 1.37824